# Technical Factor - Williams Accumulation/Distribution (WAD)

In [13]:
import dai
import pandas as pd

In [14]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [15]:
sql = f"""
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
    WHERE date BETWEEN '1990-01-01 00:00:00.000' AND '{ed} 23:59:59.999'
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        IF(close >= m_lag(close, 1), close - LEAST(low, m_lag(close, 1)), close - GREATEST(high, m_lag(close, 1))) AS WADI,
        SUM(WADI) OVER (PARTITION BY instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)    AS WAD,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(WAD > m_lag(WAD, 1), 1, 0) AS TRBY1,
        IF(WAD < m_lag(WAD, 1), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF((WAD - m_lag(WAD, 1)) > 0 AND ((WAD - m_lag(WAD, 1)) - (m_lag(WAD, 1) - m_lag(WAD, 2))) > 0, 1, 0) AS MTBY1,
        IF((WAD - m_lag(WAD, 1)) < 0 AND ((WAD - m_lag(WAD, 1)) - (m_lag(WAD, 1) - m_lag(WAD, 2))) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF((WAD - m_lag(WAD, 1)) > 0 AND (m_lag(WAD, 1) - m_lag(WAD, 2)) < 0, 1, 0) AS RVBY1,
        IF((WAD - m_lag(WAD, 1)) < 0 AND (m_lag(WAD, 1) - m_lag(WAD, 2)) > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(WAD >= m_max(WAD, 20) AND m_lag(WAD, 1) < m_lag(m_max(WAD, 20), 1), 1, 0) AS BKBY1,
        IF(WAD <= m_min(WAD, 20) AND m_lag(WAD, 1) > m_lag(m_min(WAD, 20), 1), 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        WAD,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
AND date BETWEEN '{sd} 00:00:00.000' AND '{ed} 23:59:59.999'
ORDER BY date, instrument
"""

In [16]:
df = dai.query(sql, filters={"instrument":instrument_list}).df()
df

,date,instrument,WAD,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-02,600519.SH,7289.524734,0,1,0,1,0,0,0,1
1,2025-01-03,600519.SH,7126.687982,0,1,0,0,0,0,0,0
2,2025-01-06,600519.SH,6841.581112,0,1,0,1,0,0,0,0
3,2025-01-07,600519.SH,6851.356204,1,0,1,0,1,0,0,0
4,2025-01-08,600519.SH,6980.387428,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
238,2025-12-25,600519.SH,6262.030085,1,0,1,0,1,0,0,0
239,2025-12-26,600519.SH,6219.710655,0,1,0,1,0,1,0,0
240,2025-12-29,600519.SH,6117.248643,0,1,0,1,0,0,0,0
241,2025-12-30,600519.SH,6013.519581,0,1,0,1,0,0,0,0
